<a href="https://colab.research.google.com/github/masolanoo/x-rays/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! kaggle datasets download nih-chest-xrays/data

100% 42.0G/42.0G [11:04<00:00, 78.8MB/s]
100% 42.0G/42.0G [11:04<00:00, 67.9MB/s]
